In [408]:
# import libs

import numpy as np
import Laser
import gCode
import random
import Printer


# Important definition
positive angle in CCW direction

### Units
Distances: mm  
Angles: degrees

# Define functions

In [409]:
# read G-Code file and return a list of strings
def read_gcode_file(filename):
    tmp = []
    with open(filename, 'r') as f:
        gcode = f.readlines()
    for line in gcode:
        tmp.append(line)
    
    return np.array(tmp)
    

In [410]:
def parseCoord(gCodeObj,linestring):
    line = linestring
    X = None
    Y = None
    Z = None
    move = False

    # check the command of the gcode line (string until first space)
    command = line.split(' ')[0]

    # check if line is ;LAYER:1 and gCodeObj.firstLayerCheck is False
    if line.startswith(';LAYER:1') and not gCodeObj.firstLayerCheck:
        gCodeObj.updateFirstLayerCheck(True)
        print("First layer check is True")

    # check if firstLayerCheck is True
    if gCodeObj.firstLayerCheck:
        # if command == G1 or G0 than move = True
        if command == 'G1' or command == 'G0':
            move = True
        if command == 'G1':
            gCodeObj.updateNumberOfG1()
            
        # check if line contains ; which means it is a comment and skip line
        if not ';' in line:
            # parse the X Y Z coords from string to float
            if 'X' in line:
                X = float(line.split('X')[-1].split(' ')[0])
            elif 'X' not in line:
                X = None
            if 'Y' in line:
                Y = float(line.split('Y')[-1].split(' ')[0])
            elif 'Y' not in line:
                Y = None
            if 'Z' in line:
                Z = float(line.split('Z')[-1].split(' ')[0])
            elif 'Z' not in line:
                Z = None
    else:
        X = None
        Y = None
        Z = None
    return move, X, Y, Z, command, linestring


In [411]:
# calculate the distance between two points
def calcDistance(x0, y0, x1, y1):
    return np.sqrt((x1-x0)**2 + (y1-y0)**2)


In [412]:
# calculate angle between two points (atan2)
def calcAngle(x0, y0, x1, y1):
    newAngle = np.arctan2((y1-y0),(x1-x0))
   
    # convert to degrees
    newAngle = np.rad2deg(newAngle)
    
    # normalize angle
    #newAngle = np.mod(newAngle, 360)
    
    return newAngle

In [413]:
def findClosestAngle(targetAngle, LaserObj):
    currentAngles = LaserObj.getNormAngles()
    
    # find the closest angle in the list of angles
    closestAngleIndx = np.argmin(np.abs(currentAngles - targetAngle))
    closestAngle = currentAngles[closestAngleIndx]

    # calculte delta bewteen target and closest angle
    deltaAngle = targetAngle - closestAngle

    # update the angle of the laser
    #LaserObj.updateAngles(deltaAngle)

    return closestAngleIndx, deltaAngle

    

In [414]:

def convertAngleToSteps(angle):
    return angle


In [415]:
def postProcessGcode(gCodeArray, threshold, LaserObj, newFilePath, gCodeObj):

    for i in range(gCodeArray.shape[0]-1):
        move0, X0, Y0, Z0, command0, line0 = parseCoord(gCodeObj, gCodeArray[i])
        move1, X1, Y1, Z1, command1, line1 = parseCoord(gCodeObj,gCodeArray[i+1])

        # update lastPos if move1 is False and move0 is True
        if move0 and not move1:
            gCodeObj.updateLastPos(X0, Y0, Z0)


        # clear Extruder for testing
        if(0):
            # copy line0 until reaches E 
            line0 = line0.split('E')[0] + '\n'

        # check if both lines are moves
        if move0 and move1:
            # check if the distance between the two points is larger than the threshold
        
            #print("i:", i+1, "move0: ", move0, "move1: ", move1, "X0: ", X0, "Y0: ", Y0, "Z0: ", Z0, "command0: ", command0, "line0: ", line0, "X1: ", X1, "Y1: ", Y1, "Z1: ", Z1, "command1: ", command1, "line1: ", line1)
            dist = calcDistance(X0, Y0, X1, Y1)
            gCodeObj.updateTotalLengthOfG1(dist)

            # calculate the angle between the two points
            targetAngle = calcAngle(X0, Y0, X1, Y1)
            lastAngle = gCodeObj.getLastAngle()
            
            # update the last angle
            gCodeObj.updateLastAngle(targetAngle)

                
            if dist > threshold:            # and not (abs(lastAngle - targetAngle)>45)

                # noralize the traget angle
                targetAngle = np.mod(targetAngle, 360)

                # find the closest angle to the target angle
                closestAngleIndx, deltaAngle = findClosestAngle(targetAngle, LaserObj)
                # update the angle of the laser
                LaserObj.updateAngles(deltaAngle)
                # convert the angle to steps
                steps = convertAngleToSteps(deltaAngle)
                # add new gcode line
                    # rotate the laser by delta angle
                        # for relative axis movement of laser introduce a G91
                        # reset the relative axis movement with G90

                tmpAngle = LaserObj.getAngles()[closestAngleIndx]

                # insert some lines for debugging
                debugLine = "\n \n \n; X0 " +str(X0) + " Y0: " + str(Y0)  + " X1: " + str(X1) + " Y1: " + str(Y1) + " dist: " + str(dist) + " targetAngle: " + str(targetAngle) + " closestAngle: " + str(closestAngleIndx) + " position of index: " + str(tmpAngle)  + " deltaAngle: " + str(deltaAngle) + '\n' #+ "position of index: " + LaserObj.getAngles()[closestAngleIndx] 
                debugLine += "; Line0 "+str(line0)
                debugLine += "; Line1 "+str(line1) + '\n'


                # send a string to the console M118
                debugLine += "M118" + " S\"in Line " + str(i) + " of Gcode \"" + '\n'

                # wait for 1 second
                debugLine += "G4 P1000" + '\n'

                newLine01 = "\nG91 \nG1 F6000 A" + str(steps) + '\nG90\n'
                        
                # define string offset for physical laser connection
                laserStringOffset = 2
                
                # activate laser PWM signal
                newLine02 = "M106 P" + str(closestAngleIndx+laserStringOffset) + " S0.8" +'\n'
                
                # insert line0
                newLine03 = line0
                
                # deactivate laser PWM signal
                newLine04 = "M106 P" + str(closestAngleIndx+laserStringOffset) + " S0.0"


                # check if the last activated laser is the same as the current laser
                # if true than skip the de- & activation of the laser
                #if LaserObj.getLastActivatedLaser != None:
                #    if LaserObj.getLastActivatedLaser == closestAngleIndx:

                lines = [newLine01, newLine02, newLine03, newLine04]

                # append new lines to newFilePath
                with open(newFilePath, 'a') as f:
                    f.write(debugLine)
                    f.write(newLine01)
                    f.write(newLine02)
                    f.write(newLine03)
                    f.write(newLine04)
                    f.close()
        else:
            # append line0 to newFilePath
            with open(newFilePath, 'a') as f:
                f.write(line0)
                f.close()

    return True

# Test functions

In [416]:
# test calcAngle function
if (0):
    print(calcAngle(0,0,0,0))
    print(calcAngle(0,0,1,-1))
    print(calcAngle(0,0,-1,-1))
    print(calcAngle(0,0,-1,1))


In [417]:
# test the parseCoord function
if(0):

# read the gcode file
    gcode = read_gcode_file("GCODE.txt")

    # create empty lists for the coordinates
    X = []
    Y = []
    Z = []
    cmd = []
    i = 0
    # loop over all lines in the gcode file
    for line in gcode:
        #print(i)
        #i += 1
        # parse the coordinates from the line
        moveBool, x, y, z, command, linestring = parseCoord(line)
        # append the coordinates to the lists
        X.append(x)
        Y.append(y)
        Z.append(z)
        cmd.append(command)

# print first 10 elements of X Y Z
j = 50
if(0):
    for i in range(j):
        print(f"X: {X[i]} Y: {Y[i]} Z: {Z[i]} cmd: {cmd[i]}")

# Main 

In [418]:
# initiate Printer object, for duration calculation of process
Prusa = Printer.Printer(velocity_trans=100, velocity_angular=100)

# read the gcode file
gcode = read_gcode_file("Input/PI3MK3M_Square_50mm.gcode")
gCodeObj = gCode.gCode("gCode")

# print shape of gcode
print(gcode.shape)

# create laser object
numberOfLaser = 4
laser = Laser.Laser(numberOfLaser)

# threshold for the distance between two points 
minDistance = 5

# check index in Output/index.txt
with open("Output/index.txt", 'r') as f:
    index = int(f.read())
    f.close()

# create new file
newFilePath = "Output/GCODE_postprocessed_" + str(index) + ".gcode"

# iterate index in Output/index.txt
with open("Output/index.txt", 'w') as f:
    f.write(str(index+1))
    f.close()

# post process the gcode
print(postProcessGcode(gcode, minDistance, laser, newFilePath, gCodeObj))


(157,)
First layer check is True
True


In [419]:
print(gCodeObj.firstLayerCheck)
print(laser.absLaserMovement)
print("Total lenght of G1 commands:\t " + str(gCodeObj.getTotalLengthOfG1()) + " mm")
print("Total angle turned: \t" + str(laser.getAbsLaserMovement()))
print("Total number of G1 commands: \t\t" + str(gCodeObj.getNumberOfG1()))



True
45.02895539674188
Total lenght of G1 commands:	 2437.8711843502765 mm
Total angle turned: 	45.02895539674188
Total number of G1 commands: 		158
